In [75]:
import numpy as np
import pandas as pd
import spacy
import string
import pickle as pkl
from sklearn import preprocessing

In [84]:
#load in the data
train_df = pd.read_csv('snli_train.tsv', sep="\t")
val_df = pd.read_csv('snli_val.tsv',sep="\t")

#get data & convert sentences to lists
train_sentence1=train_df['sentence1'].values.tolist()
train_sentence2=train_df['sentence2'].values.tolist()
val_sentence1=val_df['sentence1'].values.tolist()
val_sentence2=val_df['sentence2'].values.tolist()

#convert the text labels to numeric
le = preprocessing.LabelEncoder()
le.fit(train_df['label'])
train_targets=le.transform(train_df['label']).tolist()
val_targets=le.transform(val_df['label']).tolist()

In [31]:
# Load English tokenizer, tagger, parser, NER and word vectors
tokenizer = spacy.load('en_core_web_sm')
punctuations = string.punctuation

# lowercase and remove punctuation
def tokenize(sent):
  tokens = tokenizer(sent)
  return [token.text.lower() for token in tokens if (token.text not in punctuations)]

In [24]:
# This is the code cell that tokenizes train/val/test datasets
def tokenize_dataset(dataset):
    token_dataset = []
    # we are keeping track of all tokens in dataset 
    # in order to create vocabulary later
    all_tokens = []
    
    for sample in dataset:
        tokens = tokenize(sample)
        token_dataset.append(tokens)
        all_tokens += tokens

    return token_dataset, all_tokens

Tokenizing train data
Tokenizing val data


In [ ]:
#run this cell only once, otherwise leverage the existing saved files
#train set tokens
print ("Tokenizing train data")
train_sentence1_tokens, train_all_sentence1_tokens = tokenize_dataset(train_sentence1)
train_sentence2_tokens, train_all_sentence2_tokens = tokenize_dataset(train_sentence2)
pkl.dump(train_sentence1_tokens, open("train_sentence1_tokens.p", "wb"))
pkl.dump(train_sentence2_tokens, open("train_sentence2_tokens.p", "wb"))
pkl.dump(train_all_sentence1_tokens, open("train_all_sentence1_tokens.p", "wb"))
pkl.dump(train_all_sentence2_tokens, open("train_all_sentence2_tokens.p", "wb"))

#val set tokens
print ("Tokenizing val data")
val_sentence1_tokens, _ = tokenize_dataset(val_sentence1)
val_sentence2_tokens, _ = tokenize_dataset(val_sentence2)
pkl.dump(val_sentence1_tokens, open("val_sentence1_tokens.p", "wb"))
pkl.dump(val_sentence2_tokens, open("val_sentence2_tokens.p", "wb"))

In [33]:
#If you have previously run the previous cell, run this cell instead to load preprocessed datasets
train_sentence1_tokens = pkl.load(open("train_sentence1_tokens.p", "rb"))
train_sentence2_tokens = pkl.load(open("train_sentence2_tokens.p", "rb"))
train_all_sentence1_tokens = pkl.load(open("train_all_sentence1_tokens.p", "rb"))
train_all_sentence2_tokens = pkl.load(open("train_all_sentence2_tokens.p", "rb"))
val_sentence1_tokens = pkl.load(open("val_sentence1_tokens.p", "rb"))
val_sentence2_tokens = pkl.load(open("val_sentence2_tokens.p", "rb"))

In [34]:
#print information about the token datasets
# double checking
print ("Train sentence1 dataset size is {}".format(len(train_sentence1_tokens)))
print ("Train sentence2 dataset size is {}".format(len(train_sentence2_tokens)))
print ("Val sentence1 dataset size is {}".format(len(val_sentence1_tokens)))
print ("Val sentence2 dataset size is {}".format(len(val_sentence2_tokens)))

print ("Total number of tokens in sentence1 train dataset is {}".format(len(train_all_sentence1_tokens)))
print ("Total number of tokens in sentence2 train dataset is {}".format(len(train_all_sentence2_tokens)))
print ("Total number of *unique* tokens in sentence1 train dataset is {}".format(len(set(train_all_sentence1_tokens))))
print ("Total number of *unique* tokens in sentence2 train dataset is {}".format(len(set(train_all_sentence2_tokens))))


Train sentence1 dataset size is 100000
Train sentence2 dataset size is 100000
Val sentence1 dataset size is 1000
Val sentence2 dataset size is 1000
Total number of tokens in sentence1 train dataset is 1294135
Total number of tokens in sentence2 train dataset is 743372
Total number of *unique* tokens in sentence1 train dataset is 14131
Total number of *unique* tokens in sentence2 train dataset is 15225


In [37]:
#build vocabularies for sentence1 and sentence2
from collections import Counter

max_vocab_size = 10000
# save index 0 for unk and 1 for pad
PAD_IDX = 0
UNK_IDX = 1

def build_vocab(all_tokens):
    # Returns:
    # id2token: list of tokens, where id2token[i] returns token that corresponds to token i
    # token2id: dictionary where keys represent tokens and corresponding values represent indices
    token_counter = Counter(all_tokens)
    vocab, count = zip(*token_counter.most_common(max_vocab_size))
    id2token = list(vocab)
    token2id = dict(zip(vocab, range(2,2+len(vocab)))) 
    id2token = ['<pad>', '<unk>'] + id2token
    token2id['<pad>'] = PAD_IDX 
    token2id['<unk>'] = UNK_IDX
    return token2id, id2token

token2id_sentence1, id2token_sentence1 = build_vocab(train_all_sentence1_tokens)
token2id_sentence2, id2token_sentence2 = build_vocab(train_all_sentence2_tokens)

In [42]:
# Lets check the dictionary by loading random token from it
import random 

random_token_id = random.randint(0, len(id2token_sentence1)-1)
random_token = id2token_sentence1[random_token_id]

print ("Token id {} ; token {}".format(random_token_id, id2token_sentence1[random_token_id]))
print ("Token {}; token id {}".format(random_token, token2id_sentence1[random_token]))

Token id 584 ; token various
Token various; token id 584


In [45]:
# convert token to id in the dataset.  After running this cell we will have converted the word tokens to indices
def token2index_dataset(tokens_data,token2id):
    indices_data = []
    for tokens in tokens_data:
        index_list = [token2id[token] if token in token2id else UNK_IDX for token in tokens]
        indices_data.append(index_list)
    return indices_data

train_sentence1_data_indices = token2index_dataset(train_sentence1_tokens,token2id_sentence1)
train_sentence2_data_indices = token2index_dataset(train_sentence2_tokens,token2id_sentence2)
val_sentence1_data_indices = token2index_dataset(val_sentence1_tokens,token2id_sentence1)
val_sentence2_data_indices = token2index_dataset(val_sentence2_tokens,token2id_sentence2)

# double checking
print ("Train sentence1 dataset size is {}".format(len(train_sentence1_data_indices)))
print ("Train sentence2 dataset size is {}".format(len(train_sentence2_data_indices)))
print ("Val sentence1 dataset size is {}".format(len(val_sentence1_data_indices)))
print ("Val sentence2 dataset size is {}".format(len(val_sentence2_data_indices)))

Train sentence1 dataset size is 100000
Train sentence2 dataset size is 100000
Val sentence1 dataset size is 1000
Val sentence2 dataset size is 1000


In [55]:
#visualize a random sentence1 and sentence2 paired training example
rand_training_example = random.randint(0, len(train_sentence1) - 1)
print (train_sentence1_tokens[rand_training_example])
print(train_sentence1_data_indices[rand_training_example])

print (train_sentence2_tokens[rand_training_example])
print(train_sentence2_data_indices[rand_training_example])

['a', 'white', 'woman', 'carries', 'a', 'baby', 'girl', 'while', 'walking', 'on', 'grass']
[2, 20, 11, 538, 2, 164, 29, 23, 41, 6, 95]
['the', 'woman', 'is', 'crawling', 'on', 'the', 'ground']
[3, 10, 4, 2112, 8, 3, 204]


In [70]:
#Check average, max, min sentence lengths to determine word padding
total_sent1_len=0
total_sent2_len=0
sent1_lens=[]
sent2_lens=[]
for i in range(0,len(train_sentence1_tokens)):
    total_sent1_len+=len(train_sentence1_tokens[i])
    total_sent2_len+=len(train_sentence2_tokens[i])
    sent1_lens.append(len(train_sentence1_tokens[i]))
    sent2_lens.append(len(train_sentence2_tokens[i]))

avg1=total_sent1_len/len(train_sentence1)
avg2=total_sent2_len/len(train_sentence2)
print("sentence1 average is: "+str(avg1)+", max is: "+str(max(sent1_lens))+", min is: "+str(min(sent1_lens)))
print("sentence2 average is: "+str(avg2)+", max is: "+str(max(sent2_lens))+", min is: "+str(min(sent2_lens)))

sentence1 average is: 12.94135, max is: 78, min is: 2
sentence2 average is: 7.43372, max is: 38, min is: 1


In [115]:
import numpy as np
import torch
from torch.utils.data import Dataset

class NewsGroupDataset(Dataset):
    """
    Class that represents a train/validation/test dataset that's readable for PyTorch
    Note that this class inherits torch.utils.data.Dataset
    """
    
    def __init__(self, sent1_list, sent2_list, target_list, MAX_SENTENCE_LENGTH):
        """
        @param data_list: list of newsgroup tokens 
        @param target_list: list of newsgroup targets 

        """
        self.sent1_list = sent1_list
        self.sent2_list = sent2_list
        self.MAX_SENTENCE_LENGTH=MAX_SENTENCE_LENGTH
        self.target_list = target_list
        assert (len(self.sent1_list) == len(self.sent2_list) == len(self.target_list))

    def __len__(self):
        return len(self.sent1_list+self.sent2_list)
        
    def __getitem__(self, key):
        """
        Triggered when you call dataset[i]
        """
        #token_idx1 = self.sent1_list[key][:self.MAX_SENTENCE_LENGTH]
        #token_idx2 = self.sent2_list[key][:self.MAX_SENTENCE_LENGTH]
        token_idx= self.sent1_list[key][:self.MAX_SENTENCE_LENGTH] + self.sent2_list[key][:self.MAX_SENTENCE_LENGTH]
        label = self.target_list[key]
        return [token_idx, len(token_idx), label]

In [116]:
train_dataset = NewsGroupDataset(train_sentence1_data_indices,train_sentence2_data_indices,train_targets, 15)

In [119]:
for i in range(10):
    print(train_dataset[i][1])

24
22
18
23
18
17
17
15
19
20


In [120]:
## example output

print("x {};\ny {}".format(train_dataset[0][0], train_dataset[0][2]))

x [2, 22, 29, 3, 2, 83, 19, 27, 6, 2, 635, 1660, 2, 298, 9, 2, 33, 20, 64, 3, 907, 118, 3, 46];
y 2


In [121]:
MAX_SENTENCE_LENGTH=30

def newsgroup_collate_func(batch):
    """
    Customized function for DataLoader that dynamically pads the batch so that all 
    data have the same length
    """
    data_list = []
    label_list = []
    length_list = []
    #print("collate batch: ", batch[0][0])
    #batch[0][0] = batch[0][0][:MAX_SENTENCE_LENGTH]
    for datum in batch:
        label_list.append(datum[2])
        length_list.append(datum[1])
    # padding
    for datum in batch:
        padded_vec = np.pad(np.array(datum[0]), 
                                pad_width=((0,MAX_SENTENCE_LENGTH-datum[1])), 
                                mode="constant", constant_values=0)
        data_list.append(padded_vec)
    return [torch.from_numpy(np.array(data_list)), torch.LongTensor(length_list), torch.LongTensor(label_list)]

In [135]:
BATCH_SIZE = 32

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=BATCH_SIZE,
                                           collate_fn=newsgroup_collate_func,
                                           shuffle=True)

val_dataset = NewsGroupDataset(val_sentence1_data_indices, val_sentence2_data_indices, val_targets, 15)
val_loader = torch.utils.data.DataLoader(dataset=val_dataset, 
                                           batch_size=BATCH_SIZE,
                                           collate_fn=newsgroup_collate_func,
                                           shuffle=True)

In [136]:
print(type(train_loader))

<class 'torch.utils.data.dataloader.DataLoader'>


In [137]:
### checking your data loader

for i, (data, lengths, labels) in enumerate(train_loader):
    print(data)
    print(labels)
    break

IndexError: list index out of range